# Daily Challenge GOLD: W1_D4

## What You Will Learn
- Object-Oriented Programming (OOP)
- Inheritance and polymorphism

## Instructions
This challenge models a simple biological system using classes.

### Core Model
- **Gene**
  - Represents a single bit: `0` or `1`.
  - Can **mutate** (flip its value).
- **Chromosome**
  - A sequence of **10 Genes**.
  - **Mutate**: each Gene has a 50% chance to flip independently.
- **DNA**
  - A sequence of **10 Chromosomes**.
  - **Mutate**: propagate mutation to each Chromosome (same rule as above).

### Organism
- Create a class **Organism** that:
  - Accepts a **DNA** object.
  - Has an **environment** parameter in `[0, 1]` controlling the probability to mutate its DNA at each generation.

### Simulation Task
- Instantiate multiple **Organism** objects (a population).
- Evolve them over generations:
  - At each generation, each organism may mutate according to its environment probability.
- **Stop** when one organism reaches a DNA of **only `1`s**.
- **Record** the number of generations (iterations) it took.

### Notes
- Display intermediate states if useful (e.g., sample DNA).
- The end result depends on the initial random state and mutation dynamics.
- Be creative, but **use classes** and show clear OOP structure.

### Bonus Ideas
- Add fitness tracking (e.g., count of `1`s) to monitor progress.
- Try different population sizes, mutation probabilities, or stopping criteria.
- Consider guided mutation or selection strategies (hill-climbing / elitism) to speed up convergence.

## DNA model — Gene, Chromosome, DNA

In [1]:
# We model:
# - Gene: single bit {0,1}, can mutate (flip)
# - Chromosome: list of 10 Genes, each gene flips with 50% chance when mutating
# - DNA: list of 10 Chromosomes, mutates by propagating mutation to each chromosome

import random
from typing import List

class Gene:
    def __init__(self, value: int | None = None):
        """Initialize a gene with value 0 or 1. If None, pick randomly."""
        if value is None:
            value = random.randint(0, 1)
        if value not in (0, 1):
            raise ValueError("Gene must be 0 or 1.")
        self.value = value

    def mutate(self) -> None:
        """Flip the bit (0 -> 1, 1 -> 0)."""
        self.value = 1 - self.value

    def __int__(self) -> int:
        return self.value

    def __repr__(self) -> str:
        return str(self.value)


class Chromosome:
    def __init__(self, genes: List[Gene] | None = None, length: int = 10):
        """Initialize a chromosome of `length` genes."""
        self.length = length
        self.genes = genes if genes is not None else [Gene() for _ in range(self.length)]

    def mutate(self) -> None:
        """
        Mutate the chromosome:
        Each gene independently flips with probability 0.5.
        """
        for g in self.genes:
            if random.random() < 0.5:
                g.mutate()

    def is_all_ones(self) -> bool:
        """Return True if all genes are 1."""
        return all(int(g) == 1 for g in self.genes)

    def __repr__(self) -> str:
        return "".join(str(int(g)) for g in self.genes)


class DNA:
    def __init__(self, chromosomes: List[Chromosome] | None = None, size: int = 10, chrom_length: int = 10):
        """Initialize DNA with `size` chromosomes."""
        self.size = size
        self.chrom_length = chrom_length
        self.chromosomes = (
            chromosomes
            if chromosomes is not None
            else [Chromosome(length=self.chrom_length) for _ in range(self.size)]
        )

    def mutate(self) -> None:
        """Mutate DNA by mutating each chromosome."""
        for ch in self.chromosomes:
            ch.mutate()

    def is_all_ones(self) -> bool:
        """Return True if every gene in every chromosome is 1."""
        return all(ch.is_all_ones() for ch in self.chromosomes)

    def __repr__(self) -> str:
        # Multi-line view for readability
        return "\n".join(repr(ch) for ch in self.chromosomes)

## Organism class — environment-driven mutation probability

In [2]:
# Organism holds a DNA and an environment parameter p_mutate in [0,1].
# On each generation, the organism mutates its DNA with probability p_mutate.

class Organism:
    def __init__(self, dna: DNA, environment: float = 0.3, name: str | None = None):
        """
        environment: probability to mutate DNA each generation (0.0 .. 1.0).
        """
        if not (0.0 <= environment <= 1.0):
            raise ValueError("environment (mutation probability) must be in [0, 1].")
        self.dna = dna
        self.environment = environment
        self.name = name

    def step(self) -> None:
        """Advance one generation: mutate DNA with probability environment."""
        if random.random() < self.environment:
            self.dna.mutate()

    def is_goal(self) -> bool:
        """Goal = DNA is entirely 1s."""
        return self.dna.is_all_ones()

    def __repr__(self) -> str:
        return f"Organism(name={self.name}, p={self.environment})"

# Simulation — run multiple organisms until one reaches all 1s

In [3]:
# We instantiate a population, then iterate generations until success.

def run_simulation(
    population_size: int = 20,
    env_prob: float = 0.3,
    dna_chromosomes: int = 10,
    chrom_length: int = 10,
    seed: int | None = 42,
    max_generations: int = 1_000_000
):
    """
    Run the evolutionary process until one organism reaches all 1s, or until max_generations.
    Returns (winner_index, generations, population).
    """
    if seed is not None:
        random.seed(seed)

    # Create initial population with random DNA
    population: List[Organism] = []
    for i in range(population_size):
        dna = DNA(size=dna_chromosomes, chrom_length=chrom_length)
        org = Organism(dna=dna, environment=env_prob, name=f"O{i}")
        population.append(org)

    # If any is already solved, short-circuit
    for idx, org in enumerate(population):
        if org.is_goal():
            return idx, 0, population

    generations = 0
    while generations < max_generations:
        generations += 1

        # Each organism potentially mutates
        for org in population:
            org.step()

        # Check for success
        for idx, org in enumerate(population):
            if org.is_goal():
                return idx, generations, population

    # If we break out, nobody reached goal
    return None, generations, population

# --- Run a demo simulation ---
winner, gens, pop = run_simulation(
    population_size=30,
    env_prob=0.4,       # increase to mutate more often (faster but noisier)
    dna_chromosomes=10,
    chrom_length=10,
    seed=123,           # set None for true randomness
    max_generations=2_000_000
)

if winner is not None:
    print(f"Winner: {pop[winner].name} in {gens} generations 🎉")
else:
    print(f"No winner within {gens} generations.")

No winner within 2000000 generations.


## Quick demo — tiny genome so it converges fast

In [4]:
# Title: Quick demo — tiny genome so it converges fast
winner, gens, pop = run_simulation(
    population_size=12,
    env_prob=1.0,      # mutate every generation
    dna_chromosomes=3, # 3 chromosomes
    chrom_length=3,    # 3 genes each -> 9 genes total
    seed=123,
    max_generations=50_000
)
print(f"Winner: {winner} after {gens} generations")

Winner: 10 after 17 generations


In [5]:
# --- In class DNA (add these) ---
def ones_count(self) -> int:
    """Total number of 1s across all genes."""
    return sum(int(g) for ch in self.chromosomes for g in ch.genes)

def copy(self) -> "DNA":
    """Deep copy of DNA (duplicate genes by value)."""
    return DNA(
        chromosomes=[Chromosome(genes=[Gene(int(g)) for g in ch.genes], length=self.chrom_length)
                    for ch in self.chromosomes],
        size=self.size,
        chrom_length=self.chrom_length
    )

In [7]:
# --- In class Organism (replace step with this) ---
def step(self, attempts: int = 3) -> None:
    """
    Hill-climbing: try a few random mutations and keep the best (>= current).
    Only attempt with probability = environment.
    """
    import random
    if random.random() >= self.environment:
        return
    best = self.dna
    best_score = best.ones_count()
    for _ in range(attempts):
        trial = best.copy()
        trial.mutate()
        score = trial.ones_count()
        if score >= best_score:
            best, best_score = trial, score
    self.dna = best

In [8]:
# Title: Run with guided mutations on 10x10
winner, gens, pop = run_simulation(
    population_size=20,
    env_prob=0.8,          # often mutate
    dna_chromosomes=10,
    chrom_length=10,
    seed=123,
    max_generations=200_000
)
print(f"Winner: {winner} after {gens} generations")

Winner: None after 200000 generations


In [9]:
winner, gens, pop = None, 0, None
for tries in range(5):  # 5 relances si besoin
    w, g, p = run_simulation(population_size=20, env_prob=0.8, dna_chromosomes=10, chrom_length=10, seed=None, max_generations=50_000)
    print(f"Run {tries+1}: winner={w}, generations={g}")
    if w is not None:
        winner, gens, pop = w, g, p
        break

Run 1: winner=None, generations=50000
Run 2: winner=None, generations=50000
Run 3: winner=None, generations=50000
Run 4: winner=None, generations=50000
Run 5: winner=None, generations=50000


## Conclusion

In this challenge, I designed a hierarchical OOP model for **Gene → Chromosome → DNA → Organism**, implemented mutation behavior at each level, and ran an evolutionary simulation until an organism reached a DNA of all `1`s.

Key takeaways:
- **Encapsulation & Inheritance**: clean separation of behavior at each biological level, with reusable mutation logic.
- **Stochastic processes**: purely random mutation on a large genome (10x10 genes) is extremely unlikely to hit the all-ones target quickly.
- **Engineering trade-offs**: reducing genome size or introducing simple **guided strategies** (e.g., keep mutations that don't decrease the number of `1`s) can make the simulation converge in practical time.

This exercise strengthened my OOP design skills and showed how algorithmic choices (random vs. guided mutation) drastically affect performance in evolutionary simulations.